<a href="https://colab.research.google.com/github/nyp-sit/iti107/blob/main/session-3/3.fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning

Another widely used transfer learning technique is _fine-tuning_. 
Fine-tuning involves unfreezing a few of the top layers 
of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in our case, the 
fully-connected classifier) and these unfrozen top layers. This is called "fine-tuning" because it slightly adjusts the more abstract 
representations of the model being reused, in order to make them more relevant for the problem at hand.

![fine-tuning VGG16](https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/resources/it3103/vgg16_fine_tuning.png)

In [1]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

It was necessary to freeze the convolution base of VGG16 in order to be able to train a randomly initialized 
classifier on top. For the same reason, it is only possible to fine-tune the top layers of the convolutional base once the classifier on 
top has already been trained. If the classified wasn't already trained, then the error signal propagating through the network during 
training would be too large, and the representations previously learned by the layers being fine-tuned would be destroyed. Thus the steps 
for fine-tuning a network are as follow:

1. Add your custom network on top of an already trained base network.
2. Freeze the base network.
3. Train the part you added.
4. Unfreeze some layers in the base network.
5. Jointly train both these layers and the part you added.


In [19]:
img_height, img_width = 128, 128

# Load the pre-trained model 
base_model = keras.applications.VGG16(input_shape=(img_height, img_width) + (3,),
                                         include_top=False,
                                         weights='imagenet')

preprocess_input_fn = keras.applications.vgg16.preprocess_input

# freeze the base layer 
base_model.trainable = False

# Add input layer 
inputs = layers.Input(shape=(img_height, img_width, 3))
# Add preprocessing layer
x = preprocess_input_fn(inputs)
# Add the base, set training to false to freeze the convolutional base
x = base_model(x)
# Add our classification head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(rate=0.5)(x)
x = layers.Dense(units=512, activation="relu")(x)
x = layers.Dropout(rate=0.5)(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)

model = keras.models.Model(inputs=[inputs], outputs=[outputs])

base_learning_rate = 0.001

model.compile(loss="binary_crossentropy", 
                  optimizer=keras.optimizers.Adam(learning_rate=base_learning_rate), 
                  metrics=["accuracy"])


Let's confirm all the layers of convolutional base are frozen. 

In [10]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_7 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


Let's print out the model summary and see how many trainable weights. We can see that we only 263,169 trainable weights (parameters), coming from the classification head that put on top of the convolutional base. (For comparison, a VGG16 has total of 14,714,688 weights).

In [11]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_3 ( (None, 128, 128, 3)       0         
_________________________________________________________________
tf.nn.bias_add_3 (TFOpLambda (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               2626

## Creating Datasets

We will setup our training and validation dataset as we did in earlier exercise.

In [13]:
dataset_URL = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/iti107/datasets/intel_emotions_dataset.zip'
path_to_zip = tf.keras.utils.get_file('intel_emotions_dataset.zip', origin=dataset_URL, extract=True, cache_dir='.')
dataset_dir = os.path.dirname(path_to_zip)

504283136/504272804 [==============================] - 36s 0us/step


In [14]:
batch_size = 32
image_size = (img_height, img_width)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

Found 1630 files belonging to 2 classes.
Using 1304 files for training.
Found 1630 files belonging to 2 classes.
Using 326 files for validation.


## Train the classification head 

We will go ahead and train our classification head.

In [20]:
# create model checkpoint callback to save the best model checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_checkpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(train_ds, validation_data=val_ds, 
          epochs=30, callbacks=[model_checkpoint_callback])

Epoch 1/30
41/41 [==============================] - 7s 128ms/step - loss: 4.3632 - accuracy: 0.5943 - val_loss: 1.5902 - val_accuracy: 0.7117
Epoch 2/30
41/41 [==============================] - 5s 117ms/step - loss: 3.2166 - accuracy: 0.6304 - val_loss: 1.3628 - val_accuracy: 0.7117
Epoch 3/30
41/41 [==============================] - 5s 122ms/step - loss: 2.0088 - accuracy: 0.6710 - val_loss: 1.0499 - val_accuracy: 0.7086
Epoch 4/30
41/41 [==============================] - 5s 118ms/step - loss: 1.5379 - accuracy: 0.7048 - val_loss: 1.0310 - val_accuracy: 0.6871
Epoch 5/30
41/41 [==============================] - 5s 117ms/step - loss: 1.3656 - accuracy: 0.6925 - val_loss: 0.8898 - val_accuracy: 0.7178
Epoch 6/30
41/41 [==============================] - 5s 117ms/step - loss: 1.1775 - accuracy: 0.7032 - val_loss: 0.7719 - val_accuracy: 0.7209
Epoch 7/30
41/41 [==============================] - 5s 116ms/step - loss: 0.9726 - accuracy: 0.7117 - val_loss: 0.7134 - val_accuracy: 0.6994
Epoch 

Now we have our classification layers trained, let's start to unfreeze some top layers of the convolutional base to fine tune the weights. 
We will fine-tune the last 3 convolutional layers, which means that all layers up until `block4_pool` should be frozen, and the layers 
`block5_conv1`, `block5_conv2` and `block5_conv3` should be trainable.

Why not fine-tune more layers? Why not fine-tune the entire convolutional base? We could. However, we need to consider that:

* Earlier layers in the convolutional base encode more generic, reusable features, while layers higher up encode more specialized features. It is 
more useful to fine-tune the more specialized features, as these are the ones that need to be repurposed on our new problem. There would 
be fast-decreasing returns in fine-tuning lower layers.
* The more parameters we are training, the more we are at risk of overfitting. The convolutional base has 15M parameters, so it would be 
risky to attempt to train it on our small dataset.

Thus, in our situation, it is a good strategy to only fine-tune the top 2 to 3 layers in the convolutional base.

Let's set this up, we will unfreeze our `base_model`, 
and then freeze individual layers inside of it, except the last 3 layers. 

Do a model ``summary()`` and you will see now that the number of trainable weights are now 7,079,424 (around 7 millions), much less than previously, because all the layers are frozen except the last 3 layers.

In [21]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

Let us examine model summary again. We can see now that we have more trainable weights 7,342,593 compared to previously 263,169.

In [22]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_4 ( (None, 128, 128, 3)       0         
_________________________________________________________________
tf.nn.bias_add_4 (TFOpLambda (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 512)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               2626

As you are training a much larger model and want to readapt the pretrained weights, it is important to use a lower learning rate at this stage as we do not want to make too drastic changes to the weights in the convolutional layers under fine-tuning.

In [23]:
finetune_learning_rate = base_learning_rate / 10.

model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=finetune_learning_rate),
              metrics=["accuracy"])

model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=[model_checkpoint_callback])

Epoch 1/30
41/41 [==============================] - 9s 144ms/step - loss: 0.6146 - accuracy: 0.7454 - val_loss: 0.6338 - val_accuracy: 0.6534
Epoch 2/30
41/41 [==============================] - 6s 133ms/step - loss: 0.5419 - accuracy: 0.7285 - val_loss: 0.6226 - val_accuracy: 0.6840
Epoch 3/30
41/41 [==============================] - 6s 134ms/step - loss: 0.4462 - accuracy: 0.7991 - val_loss: 0.6021 - val_accuracy: 0.7147
Epoch 4/30
41/41 [==============================] - 6s 137ms/step - loss: 0.3599 - accuracy: 0.8466 - val_loss: 0.6141 - val_accuracy: 0.7117
Epoch 5/30
41/41 [==============================] - 6s 135ms/step - loss: 0.3112 - accuracy: 0.8781 - val_loss: 0.5987 - val_accuracy: 0.7362
Epoch 6/30
41/41 [==============================] - 6s 134ms/step - loss: 0.2163 - accuracy: 0.9126 - val_loss: 0.7211 - val_accuracy: 0.7239
Epoch 7/30
41/41 [==============================] - 6s 135ms/step - loss: 0.1975 - accuracy: 0.9179 - val_loss: 0.6623 - val_accuracy: 0.7055
Epoch 

In [24]:
model.load_weights('best_checkpoint')
model.evaluate(val_ds)

11/11 [==============================] - 1s 82ms/step - loss: 0.6027 - accuracy: 0.7362


[0.6026899218559265, 0.7361963391304016]

**Question:**

Is our fine-tuned model performing better or worse? 

Provide a possible explanation to your observation. 

Suggest one experiment you can try to improve your model.

<details><summary>Click here for answer</summary>
It seems that our model quickly overfits, and the validation accuracy is not any better than the previous non-finetuned model. 

It may be because our training set is rather small, and the unfrozen layers have a lot more parameters to train, thus leading to overfitting.

We can try to unfreeze less layers.
</details>